In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
import pandas_profiling as pp

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split

In [ ]:
data = pd.read_csv('../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')
data.head(5)

## EDA

In [ ]:
data.info()

In [ ]:
#RP = ProfileReport(data)
#RP.to_file(output_file='output.html')

In [ ]:
%matplotlib inline

data.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
corr_matrix=data.corr()

corr_matrix['quality'].sort_values(ascending=False)

In [ ]:
import seaborn as sns
f, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr_matrix, mask=np.zeros_like(corr_matrix, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax)

**fixed acidity have linear relation ship with density, pH and citric acid**

In [ ]:
plt.scatter(data['fixed acidity'], data['density'])
plt.xlabel("fixed acidity")
plt.ylabel("density")
plt.show()

In [ ]:
plt.scatter(data['fixed acidity'], data['citric acid'])
plt.xlabel("fixed acidity")
plt.ylabel("citric acid")
plt.show()

In [ ]:
plt.scatter(data['fixed acidity'], data['pH'])
plt.xlabel("fixed acidity")
plt.ylabel("pH")
plt.show()

In [ ]:
X= data.iloc[ : , :-1]

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

In [ ]:
calc_vif(X)

**pH is correlated with acidity, lower ph value mean more acidic property, we can look at vif of varible it also confirm pH can be explained by other variable present so i am going to drop ph**

In [ ]:
data.columns

In [ ]:
X=data[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
        'sulphates', 'alcohol']]

In [ ]:
# converting the response variables(3-7) as binary response variables that is either good or bad
bins = (2,6.5,8)
labels = ['bad','good']
data['quality'] = pd.cut(data['quality'],bins=bins,labels=labels)

#encoding categorical
le = LabelEncoder()
data['quality'] = le.fit_transform(data['quality'])

#set dependent variable
y=data[['quality']]

In [ ]:
data['quality'].value_counts()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

# determining the shapes of training and testing sets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform (X_test)

## Logistic Regression

In [ ]:
# creating the model
model = LogisticRegression()

# feeding the training set into the model
model.fit(X_train, y_train)

# predicting the results for the test set
y_pred = model.predict(X_test)

# calculating the training and testing accuracies
print("Training accuracy :", model.score(X_train, y_train))
print("Testing accuracy :", model.score(X_test, y_test))

# classification report
print(classification_report(y_test, y_pred))

# confusion matrix
print(confusion_matrix(y_test, y_pred))

In [ ]:
#roc_auc  score for logistic model

fpr1, tpr1, thresholds1 = roc_curve(y_test, y_pred)
auc_score1 = roc_auc_score(y_test, y_pred)

## k-nearest neighbour

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

# calculating the training and testing accuracies
print("Training accuracy :", model.score(X_train, y_train))
print("Testing accuracy :", model.score(X_test, y_test))

# classification report
print(classification_report(y_test, y_pred))

# confusion matrix
print(confusion_matrix(y_test, y_pred))

In [ ]:
#roc_auc  score for KNN model

fpr2, tpr2, thresholds2 = roc_curve(y_test, y_pred)
auc_score2 = roc_auc_score(y_test, y_pred)

In [ ]:
#comparing roc_auc_socre for bothe the model
print(auc_score1, auc_score2)

## Result

**KNN work better thn logistic regression**
